<h1 align="center">Modelos de Machine Learning para Séries Temporais</h1>
<p align="center"><img src=https://ichi.pro/assets/images/max/724/0*k2bCMOixbQLU0UDA length=300, width=450></p>

<p align="justify">Neste tópico, passaremos pelos métodos utilizados como linha de base <i>Baseline</i> ou que se destaca pelo desempenho ou facilidade de uso ou aplicabilidade. Os algoritmos que veremos são considerados pelo seu alto desempenho em termos de previsão.</p>

<h2>Métodos e Bibliotecas de Machine Learning para Séries Temporais</h2>
<p align ="justify">Ao longo dos estudos, foram apresentadas os seguintes algoritmos: SilverKite, gradient boosting e K-nearst neighbors. Recentemente, a biblioteca Kats do Facebook e a Greykite ganharam destaque por conta da sua performance e desempenho nas tarefas de predição. Porém, a TSFresh e a SkTime constam como 1º e 2º lugar, respectivamente, dentre todas as bibliotecas lançadas até o momento. Uma questão importante a ser considerada é a Validação, que veremos em seguida.</p>

<h2>Validação</h2>
<p align = "justify">Muitas vezes, para realizar as tarefas de Aprendizado de Máquina, utilizando a Validação Cruzada <i>K-Fold</i>, onde a divisão dos dados é realizada de forma pseudo-aleatória, de forma que os conjuntos de treinamento e o teste/validação possam ser de qualquer parte dos dados, desde que não tenham sido usados para o treinamento. Com dados de séries temporais, nos utilizaremos para o treinamento os dados anteriores e o testaremos na fatia de dados de mais novos. Isso nos dará uma visão mais realista do desempenho uma vez que o modelo for implementado.</p>
<p align="justify">Na validação <i>Walk-Forward</i> treinaremos um segmento inicial dos dados e depois testaremos em um período após o conjunto de treinamento. Em seguinda, avançaremos e repetiremos o processo. Desta forma, teremos um período fora da amostra e poderemos combinar os resultados sobre esses períodos. Com o <i>Walk-Forward</i>, é menos provável que ocorra o <i>Overfitting<i/>.</p>

<h2>KNN com a Distorção do Tempo Dinâmico</h2>
<p align="justify">O KNN é um método de Aprendizado de Máquina bem conhecido. Com ele, podemos obter a distância dos pontos de dados semelhantes.Então, podemos pegar os rótulos conhecidos desses vizinhos mais próximos como saída e integrá-los de alguma forma usando uma função. </p>
<p align="center"><img src=https://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/KnnClassification.svg/279px-KnnClassification.svg.png?20101222011435></p>
<p align="center">KNN para classificação</p>
<p align="justify">Neste caso, nós já conhecemos alguns pontos. Na ilustração anterior, esses pontos são indicados como quadrados e triângulos, representando pontos de dados de duas classes, respectivamente. Dado um novo ponto (círculo), nós encontramos o ponto mais próximo dos dados conhecidos que apontam para ele. No caso acima, podemos supor que o círculo pertence à classe triângulo também.</p>
<p align="justify">Embora, conceitualmente simples, este método muitas vezes serve como linha de base ou até de forma bem competitiva com outros algoritmos de aprendizagem de máquina mais sofisticados, mesmo quando comparado ao KNN (k=1). Os hiperparâmetros mais importantes deste algoritmo são:</p>
<li>O número de vizinho (k) que desejamos como base na nossa saída</li>
<li>A função de integração (valor médio ou mais frequente)</li>
<li>A função de distância que é usado para encontrar os pontos de dados.</li>

<p align="justify">A distorçar de Tempo dinâmico é uma medida que é utilizada para comparar a similaridade (ou equivalência, a distância) entre duas séries temporais. Nós utilizamos o KNN em combinação com Distorção de Tempo Dinâmico (DTW) como medida de distância para encontrar séries temporais similares. Este método provou ser difícil de vencer, embora outros métodos <i>state of art</i> já o tenha superado.</p>

<h2>Silverkite</h2>
<p align="justify">O algoritmo Silverkite vêm junto com a biblioteca Greykite lançada pela LinkdIn. De acordo com o LinkdIn, ele pode lidar com diferentes tipos de tendências e sazonalidades, como eventos de hora em hora, diários, semanais, repetidos e feriados, além de efeitos de curto alcance. Dentro do LinkdIn, ele é usado tanto para prazo curto, quanto para prazo longo, com horizontes de previsão como 1 ano a frente.</p>
<p align="justify">Silverkite também constrói variáveis indicadoras para feriados. Feriados podem ser especificados por nome ou por país, ou ainda, pode ser totalmente personalizado. Os pontos de mudança podem ser especificados manualmente ou os candidatos podem ser detectados automaticamente com um modelo de regressão e, posteriomente, selecionado usando o algoritmo de LASSO. Além de tendência, sazonalidade e feriados, Silverkite incluiu um termo que é calculado com base em médias em janelas, em vez de levar atrasos (<i>lags</i>) independentemente.</p>

<h2>Gradiente boosting</h2>
<p align="justify">O Gradient Boosted Tree é um conjunto de árvores, semelhante às Florestas Aleatórias (Random Forest), no entando, como este é um algoritmo de reforço, cada árvore é calculada para reduzir incrementalmente o erro. A cada nova iteração uma árvore é escolhida e sua previsão é adicionada às previsões anteriores com base nos pesos. Há um prazo de regularização que penaliza a complexidade e reduz o <i>overfitting</i>, semelhante ao <i>Regularized Greedy Forest (GDE)</i>.</p>
<p align="justify">Especialmente, em casos de regressãos, o <i>XGBoost</i> pode ser usado para previsão. Neste cenário, o modelo é treinado com base em valores passados para prever valores futuros, e isso pode ser
aplicado a séries temporais univariadas e multivariadas.</p>

<h2>KNN com Dynamic Time Warping no Python</h2>

In [19]:
import pandas as pd
pd.read_csv("data/Computers/Computers_TRAIN.txt", sep='\t', header=None, engine='python')

,0
0,1.0000000e+00 -3.3503182e-01 -3.3503182e-...
1,1.0000000e+00 -6.0252494e-01 -6.0252494e-...
2,1.0000000e+00 -3.2022150e-01 -3.2022150e-...
3,1.0000000e+00 -2.1291650e-01 -2.1291650e-...
4,1.0000000e+00 -1.8580535e-01 -1.8580535e-...
...,...
245,2.0000000e+00 2.8199230e+00 2.2530252e+...
246,2.0000000e+00 -1.4737218e-01 -1.4737218e-...
247,2.0000000e+00 -1.4388754e-01 -4.7382074e-...
248,2.0000000e+00 -4.1048285e-01 -6.4030218e-...
